In [1]:
import pyranges as pr
import pandas as pd
from cerberus.cerberus import *

In [2]:
gtf = 'tests/files/Canx.gtf' 
mode = 'tss'
dist = 50
slack = 50

In [3]:
bed = get_ends_from_gtf(gtf, 'tss', dist, slack)

In [4]:
df = get_ics_from_gtf(gtf)

## New assign triplets

In [5]:
# def add_triplets(gtf, ic_file, tss_bed, tes_bed):
#     """
#     Determines which tss, intron chain, and tes are used from a cerberus 
#     reference are used in a specific gtf.
    
#     Parameters:
#         gtf (str): File path to gtf you want to assign triplets to
#         ic_file (str): File path to intron chain tsv (part of cerberus ref.)
#         tss_bed (str): File path to tss bed (part of cerberus ref.)
#         tes_bed (str): File path to tes bed (part of cerberus ref.)
    
#     Returns:
#         df (pandas DataFrame): File that maps each transcript from gtf to 
#             a tss, intron chain, and tes given by the cerberus reference.
#             Also includes new transcript ids to refer to each transcript
#     """
#     t_df = pr.read_gtf(gtf)
#     t_df = t_df.df
#     t_df['gene_id'] = t_df.gene_id.str.split('.', n=1, expand=True)[0]
#     t_df = pr.PyRanges(t_df)

#     ### intron chain annotation ###

#     # get unique intron chains from gtf
#     df = t_df.copy()
#     df = get_ic(df)
#     df.rename({'ic': 'Coordinates'},
#               axis=1, inplace=True)

#     # read in reference set of intron chains and format them as strings 
#     # to enable merging
#     ref = read_ic_ref(ic_file)

#     # merge on intron chain, strand, chromosome, and gene id
#     df = df.merge(ref, how='left',
#                  on=['Chromosome', 'Strand', 
#                      'Coordinates', 'gene_id'])

#     # formatting
#     df.rename({'Name': 'ic_id'}, axis=1, inplace=True)
#     df = df[['transcript_id', 'ic', 'ic_id']]

#     ### end annotation ###

#     for mode, bed_file in zip(['tss', 'tes'], [tss_bed, tes_bed]):

#         if mode == 'tss':
#             ends = t_df.features.tss()
#         elif mode == 'tes':
#             ends = t_df.features.tes()

#         # merge transcriptome ends with reference ends
#         ref = read_end_ref(bed_file, mode=mode)
#         ref = pr.PyRanges(ref)
#         ends = ends.join(ref, 
#                          strandedness='same', 
#                          how='left').df
#         ends = ends.loc[ends.gene_id == ends.gene_id_b]

#         # formatting 
#         ends.rename({'Name': '{}_id'.format(mode)}, axis=1, inplace=True)
#         ends = ends[['transcript_id', mode, '{}_id'.format(mode)]]

#         # add ends into map df
#         df = df.merge(ends, how='left', on='transcript_id')

#     ### creating map file ###

#     # get gene id / name and transcript name from original gtf
#     t_df = t_df.df
#     t_df = t_df.loc[t_df.Feature == 'transcript']
#     if 'gene_name' not in t_df.columns:
#         t_df['gene_name'] = t_df.transcript_name.str.split('-', n=1, expand=True)[0]
#     t_df = t_df[['gene_id', 'gene_name', 
#                  'transcript_id', 'transcript_name']]
#     df = df.merge(t_df, how='left', on='transcript_id')

#     # create triplets and rename old ids
#     df.rename({'transcript_id': 'original_transcript_id', 
#                'transcript_name': 'original_transcript_name'},
#                axis=1, inplace=True)
#     df['transcript_triplet'] = '['+df.tss.astype(str)+', '+\
#                                    df.ic.astype(str)+', '+\
#                                    df.tes.astype(str)+']'
#     df['transcript_id'] = df['gene_id']+' '+df.transcript_triplet
#     df['transcript_name'] = df['gene_name']+' '+df.transcript_triplet
    
#     return df

In [6]:
gtf = 'tests/files/Canx.gtf'
ic_file = 'tests/files/Canx_ics.tsv'
tss_bed = 'tests/files/Canx_tss.bed'
tes_bed = 'tests/files/Canx_tes.bed'
h5 = True

In [7]:
df = add_triplets(gtf, ic_file, tss_bed, tes_bed)

In [8]:
df

,original_transcript_id,ic,ic_id,tss,tss_id,tes,tes_id,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name
0,ENSMUST00000020637.8,3,ENSMUSG00000020368_3,1,ENSMUSG00000020368_1,2,ENSMUSG00000020368_2,ENSMUSG00000020368,Canx,Canx-001,"[1, 3, 2]","ENSMUSG00000020368 [1, 3, 2]","Canx [1, 3, 2]"
1,ENSMUST00000146979.1,5,ENSMUSG00000020368_5,4,ENSMUSG00000020368_4,4,ENSMUSG00000020368_4,ENSMUSG00000020368,Canx,Canx-003,"[4, 5, 4]","ENSMUSG00000020368 [4, 5, 4]","Canx [4, 5, 4]"
2,ENSMUST00000153068.1,4,ENSMUSG00000020368_4,3,ENSMUSG00000020368_3,5,ENSMUSG00000020368_5,ENSMUSG00000020368,Canx,Canx-002,"[3, 4, 5]","ENSMUSG00000020368 [3, 4, 5]","Canx [3, 4, 5]"
3,ENSMUST00000155801.1,2,ENSMUSG00000020368_2,2,ENSMUSG00000020368_2,3,ENSMUSG00000020368_3,ENSMUSG00000020368,Canx,Canx-004,"[2, 2, 3]","ENSMUSG00000020368 [2, 2, 3]","Canx [2, 2, 3]"
4,ENSMUST00000155801.2,2,ENSMUSG00000020368_2,2,ENSMUSG00000020368_2,3,ENSMUSG00000020368_3,ENSMUSG00000020368,Canx,Canx-004,"[2, 2, 3]","ENSMUSG00000020368 [2, 2, 3]","Canx [2, 2, 3]"
5,ENSMUST00000179865.6,1,ENSMUSG00000020368_1,1,ENSMUSG00000020368_1,1,ENSMUSG00000020368_1,ENSMUSG00000020368,Canx,Canx-201,"[1, 1, 1]","ENSMUSG00000020368 [1, 1, 1]","Canx [1, 1, 1]"
